In [7]:
import pygame
import math
import os
pygame.init()

WIDTH, HEIGHT = 900, 600
TOOLBAR_HEIGHT = 60
BUTTON_SIZE = 50
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Paint App")

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW=(255, 255, 0)

canvas = pygame.Surface((WIDTH, HEIGHT - TOOLBAR_HEIGHT))
canvas.fill(WHITE)

# Define available tools and colors
tools = ["free_draw", "eraser", "circle", "rectangle", "square", "right_triangle", "equilateral_triangle", "rhombus"]
colors = [BLACK, RED, GREEN, BLUE]
selected_tool = "free_draw"
selected_color = BLACK
drawing = False
start_pos = None
last_pos = None

# Load tool icons if available
tool_icons = {}
for tool in tools:
    path = f"{tool}.png"
    if os.path.exists(path):
        tool_icons[tool] = pygame.image.load(path)
        tool_icons[tool] = pygame.transform.scale(tool_icons[tool], (BUTTON_SIZE, BUTTON_SIZE))
    else:
        tool_icons[tool] = None  

def draw_toolbar():
    pygame.draw.rect(screen, GRAY, (0, 0, WIDTH, TOOLBAR_HEIGHT))
    
    # Draw tool buttons
    for i, tool in enumerate(tools):
        color = YELLOW if tool == selected_tool else WHITE
        pygame.draw.rect(screen, color, (10 + i * (BUTTON_SIZE + 10), 5, BUTTON_SIZE, BUTTON_SIZE))
        if tool_icons[tool]:
            screen.blit(tool_icons[tool], (10 + i * (BUTTON_SIZE + 10), 5))
    
    # Draw color selection buttons
    for i, color in enumerate(colors):
        pygame.draw.rect(screen, color, (WIDTH - (i + 1) * (BUTTON_SIZE + 10), 5, BUTTON_SIZE, BUTTON_SIZE))

def draw_smooth_line(surface, color, start, end, radius):
    # Draw a smoothly connected line by interpolating points
    dx, dy = end[0] - start[0], end[1] - start[1]
    distance = max(abs(dx), abs(dy))
    if distance == 0:
        return
    for i in range(distance):
        progress = i / distance
        x, y = int(start[0] + progress * dx), int(start[1] + progress * dy)
        pygame.draw.circle(surface, color, (x, y), radius)

def draw_square(surface, color, start, end):
    # Draw a square using the smallest side length
    size = min(abs(end[0] - start[0]), abs(end[1] - start[1]))
    pygame.draw.rect(surface, color, (start[0], start[1], size, size), 2)

def draw_right_triangle(surface, color, start, end):
    # Draw a right triangle with one right angle at 'start'
    pygame.draw.polygon(surface, color, [start, (start[0], end[1]), end], 2)

def draw_equilateral_triangle(surface, color, start, end):
    # Draw an equilateral triangle based on 'start' and 'end'
    side = abs(end[0] - start[0])
    height = (math.sqrt(3) / 2) * side
    pygame.draw.polygon(surface, color, [
        (start[0], start[1] + height),
        (start[0] + side, start[1] + height),
        (start[0] + side // 2, start[1])
    ], 2)

def draw_rhombus(surface, color, start, end):
    # Draw a rhombus centered between 'start' and 'end'
    width, height = abs(end[0] - start[0]), abs(end[1] - start[1])
    cx, cy = (start[0] + end[0]) // 2, (start[1] + end[1]) // 2
    pygame.draw.polygon(surface, color, [
        (cx, start[1]), (end[0], cy), (cx, end[1]), (start[0], cy)
    ], 2)

running = True
while running:
    screen.fill(WHITE)
    screen.blit(canvas, (0, TOOLBAR_HEIGHT))
    draw_toolbar()
    pygame.display.flip()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        
        if event.type == pygame.MOUSEBUTTONDOWN:
            x, y = event.pos
            # Check if the click is in the toolbar
            if y <= TOOLBAR_HEIGHT:
                for i, tool in enumerate(tools):
                    if 10 + i * (BUTTON_SIZE + 10) <= x <= 10 + (i + 1) * (BUTTON_SIZE + 10):
                        selected_tool = tool
                for i, color in enumerate(colors):
                    if WIDTH - (i + 1) * (BUTTON_SIZE + 10) <= x <= WIDTH - i * (BUTTON_SIZE + 10):
                        selected_color = color
            else:
                start_pos = (x, y - TOOLBAR_HEIGHT)
                drawing = True
                last_pos = start_pos

        if event.type == pygame.MOUSEBUTTONUP and start_pos:
            # Handle shape drawing on mouse release
            end_pos = (event.pos[0], event.pos[1] - TOOLBAR_HEIGHT)
            if selected_tool == "circle":
                center, radius = start_pos, max(abs(end_pos[0] - start_pos[0]), abs(end_pos[1] - start_pos[1]))
                pygame.draw.circle(canvas, selected_color, center, radius, 2)
            elif selected_tool == "rectangle":
                rect = pygame.Rect(start_pos, (end_pos[0] - start_pos[0], end_pos[1] - start_pos[1]))
                pygame.draw.rect(canvas, selected_color, rect, 2)
            elif selected_tool == "square":
                draw_square(canvas, selected_color, start_pos, end_pos)
            elif selected_tool == "right_triangle":
                draw_right_triangle(canvas, selected_color, start_pos, end_pos)
            elif selected_tool == "equilateral_triangle":
                draw_equilateral_triangle(canvas, selected_color, start_pos, end_pos)
            elif selected_tool == "rhombus":
                draw_rhombus(canvas, selected_color, start_pos, end_pos)
            start_pos, drawing, last_pos = None, False, None

    # Handle continuous drawing for free draw and eraser tools
    if pygame.mouse.get_pressed()[0] and drawing:
        pos = pygame.mouse.get_pos()
        pos = (pos[0], pos[1] - TOOLBAR_HEIGHT)
        if selected_tool == "free_draw":
            if last_pos is not None:
                draw_smooth_line(canvas, selected_color, last_pos, pos, 5)
            last_pos = pos
        elif selected_tool == "eraser":
            if last_pos is not None:
                draw_smooth_line(canvas, WHITE, last_pos, pos, 10)
            last_pos = pos

pygame.quit()
